This notebook covers the basics of operations on Series and DataFrames. Again, a quick check on whether you know the material.

1. Can you use numpy ufuncs (e.g. `np.exp`) on DataFrames?
2. What happens when you add two DataFrames with different indicies?

If you know the answers already, you can skip to our first [application](#Application:-Economic-Timeseries). Maybe download some additional datasets from [FRED](https://research.stlouisfed.org) and play with those; try to figure out how to fix the economy. You've got about 5 minutes.

`DataFrame`s and `Series` support all the usual math operations. Additionally,
numpy [`ufuncs`](http://docs.scipy.org/doc/numpy/reference/ufuncs.html) (universial function, e.g. `np.log`) can  be used as expected.

In [1]:
import numpy as np
import pandas as pd

from ph2t import side_by_side  # came with the notebooks

# `ufuncs`

`ufunc`s operate on arrays elementwise.
They can be used just like you'd expect on a `Series` or `DataFrame`.

In [2]:
np.random.seed(42)
df = pd.DataFrame(np.random.uniform(0, 10, size=(3, 3)))
df

,0,1,2
0,3.745401,9.507143,7.319939
1,5.986585,1.560186,1.559945
2,0.580836,8.661761,6.011150


In [3]:
df + 1

,0,1,2
0,4.745401,10.507143,8.319939
1,6.986585,2.560186,2.559945
2,1.580836,9.661761,7.011150


In [4]:
df ** 2

,0,1,2
0,14.028030,90.385769,53.581513
1,35.839198,2.434182,2.433429
2,0.337371,75.026112,36.133926


In [5]:
np.log(df)

,0,1,2
0,1.320529,2.252043,1.990602
1,1.789521,0.444805,0.444651
2,-0.543287,2.158918,1.793616


For any decent-sized `DataFrame`, using a `ufunc` will be much faster than applying the function to each row in a for loop or by using `DataFrame.apply`.

# Alignment

When you do binary operations between two `numpy` arrays, like `+`, the arrays are in some sense aligned by position.
It's up to you to ensure that the correct values are in the same position across arrays.

In pandas, we rely on row and column *labels* to do the alignment for us.
Pandas automatically aligns by label when doing operations between `DataFrames` and `Series`.

In [7]:
side_by_side?

In [8]:
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [2, 4, 6], 'B': [1, 4, 9]})

side_by_side(df1, df2, "df1", "df2")

,A,B
0,1,4
1,2,5
2,3,6
,A,B
0,2,1
1,4,4
2,6,9


In [9]:
df1 + df2

,A,B
0,3,5
1,6,9
2,9,15


In [10]:
# Note the index order
df2 = pd.DataFrame({'A': [6, 2, 4], 'B': [9, 1, 4]}, index=[2, 0, 1])
side_by_side(df1, df2, 'df1', 'df2')

,A,B
0,1,4
1,2,5
2,3,6
,A,B
2,6,9
0,2,1
1,4,4


In [11]:
df1 + df2

,A,B
0,3,5
1,6,9
2,9,15


In [12]:
# Different index entirely
df3 = pd.DataFrame(np.random.randn(5, 3), columns=['A', 'B', 'C'])
side_by_side(df1, df3, 'df1', 'df3')

In [13]:
df1 + df3

,A,B,C
0,0.419122,3.474830,NaN
1,1.075917,2.387451,NaN
2,3.816445,4.476124,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


# Recap

- `ufuncs` work just fine
- Operations **align by label** and then are performed

# Application: Economic Timeseries

In [24]:
from itertools import chain

import pandas as pd

import seaborn as sns
import statsmodels.api as sm
from pandas_datareader import data

import matplotlib.pyplot as plt
from toolz import partitionby as partitionby

%matplotlib inline
pd.options.display.max_rows = 10
sns.set_style('ticks')

ModuleNotFoundError: No module named 'pandas_datareader'

Pandas recently split off some web-based data reading functionality into its own package `pandas-datareader`. Earlier, I used it to grab data from [FRED](http://research.stlouisfed.org).

```python
gdp = data.DataReader("GDP", data_source='fred',
                      start='1929', end='2014').squeeze()
cpi = data.DataReader("CPIAUCSL", data_source='fred',
                      start='1947-01', end='2015-05').squeeze()
rec = data.DataReader('USREC', data_source='fred',
                      start='1854-12-01',
                      end='2014-08-01').squeeze()
gdp.to_csv('data/gdp.csv', header=True)
cpi.to_csv('data/cpi.csv', header=True)
rec.to_csv('data/rec.csv', header=True)
```

In [25]:
with open('data/gdp.csv') as f:
    print(''.join(f.readlines(100)))

DATE,GDP
1947-01-01,243.1
1947-04-01,246.3
1947-07-01,250.1
1947-10-01,260.3
1948-01-01,266.2
1948-04-01,272.9



In [26]:
pd.read_csv?

In [27]:
gdp = pd.read_csv('data/gdp.csv', index_col='DATE', parse_dates=['DATE']).squeeze()
cpi = pd.read_csv('data/cpi.csv', index_col='DATE', parse_dates=['DATE']).squeeze()
rec = pd.read_csv('data/rec.csv', index_col='DATE', parse_dates=['DATE']).squeeze()

In [28]:
gdp.head(n=5)

DATE
1947-01-01    243.1
1947-04-01    246.3
1947-07-01    250.1
1947-10-01    260.3
1948-01-01    266.2
Name: GDP, dtype: float64

In [18]:
gdp.tail()

DATE
2013-01-01    16475.4
2013-04-01    16541.4
2013-07-01    16749.3
2013-10-01    16999.9
2014-01-01    17025.2
Name: GDP, dtype: float64

In [19]:
cpi.head()

DATE
1947-01-01    21.48
1947-02-01    21.62
1947-03-01    22.00
1947-04-01    22.00
1947-05-01    21.95
Name: CPIAUCSL, dtype: float64

In [20]:
rec.head()

DATE
1854-12-01    1
1855-01-01    0
1855-02-01    0
1855-03-01    0
1855-04-01    0
Name: USREC, dtype: int64

## Methods

DataFrames and Series have a bunch of useful aggregation methods, `.mean`, `.max`, `.std`, etc.

In [21]:
gdp.mean()

5072.864684014868

## Plotting

DataFrames and Series can make decent looking plots with a few lines of code.

I wanted to incude bars for recession indicators.
The only pandas-relavent bit is `Series.iteritems`, which you *almost* never want to use since it's relatively slow.

In [29]:
next(rec.iteritems())  

(Timestamp('1854-12-01 00:00:00'), 1)

In [30]:
groups = partitionby(lambda x: x[1] == 1, rec.iteritems())
recessions = filter(lambda x: x[0][1] == 1, groups)
spans = [(months[0][0], months[-1][0]) for months in recessions]

NameError: name 'partitionby' is not defined

In [31]:
def add_rec_bars(ax=None):
    ax = ax or plt.gca()
    ylim = ax.get_ylim()
    for span in spans:
        ax.fill_between(span, *ax.get_ylim(), color='k', alpha=.25)
    ax.set_ylim(*ylim)
    return ax

sns.set(style="white", context="talk")
plt.rcParams['figure.figsize'] = (12, 6)

NameError: name 'plt' is not defined

In [ ]:
ax = gdp.plot(title='GDP', linewidth=3)
add_rec_bars(ax)
sns.despine()

Let's put some of those operations to use.

<div class="alert alert-success">
    <b>EXERCISE</b>: Convert CPI to base 2009
</div>

CPI is the Consumer Price Index. The Index part just means that it doesn't really have any units,
just some (arbitrary) time span that is set to 100, and every other observation is relative to that time.
The CPI we have is indexed to the average over 1982-1984.

**Convert the CPI base-2009 by dividing the entire Series `cpi` by the average CPI in 2009.**

Try breaking the problem into pieces:
1. Select just the rows from 2009 (`.loc`)
  + Timeseries (dates in the index) have [speical rules](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#datetimeindex-partial-string-indexing) for slicing. Pass in a string with the subset you want
    + `.loc['2010-01-01']` select all observations from that day
    + `.loc['2010-01']` selects all observations from that month
2. calculate the average on those rows
3. divide `cpi` by 100 * that number (assign that to `cpi09`)

In [40]:
# Your code goes here
# Assign the result to cpi09

cpi.loc['2009'].mean()
cpi09 = cpi/100*cpi.loc['2009'].mean()
cpi09.head()

DATE
1947-01-01    46.088490
1947-02-01    46.388881
1947-03-01    47.204227
1947-04-01    47.204227
1947-05-01    47.096944
Name: CPIAUCSL, dtype: float64

In [47]:
# %load solutions/solutions_operations.py
# CPI (base 2009)
base09 = cpi.loc['2009'].mean()
cpi09 = 100 * (cpi / base09)
cpi09.head()
cpi09.dropna()


DATE
1947-01-01     10.010968
1947-02-01     10.076216
1947-03-01     10.253319
1947-04-01     10.253319
1947-05-01     10.230016
1947-06-01     10.290604
1947-07-01     10.360513
1947-08-01     10.439743
1947-09-01     10.644809
1947-10-01     10.677434
1947-11-01     10.747343
1947-12-01     10.910464
1948-01-01     11.036300
1948-02-01     11.031639
1948-03-01     10.952409
1948-04-01     11.101548
1948-05-01     11.190100
1948-06-01     11.255348
1948-07-01     11.371863
1948-08-01     11.385845
1948-09-01     11.353221
1948-10-01     11.329918
1948-11-01     11.260009
1948-12-01     11.208742
1949-01-01     11.190100
1949-02-01     11.143494
1949-03-01     11.143494
1949-04-01     11.148154
1949-05-01     11.143494
1949-06-01     11.148154
                 ...    
2012-12-01    107.786619
2013-01-01    107.958595
2013-02-01    108.594301
2013-03-01    108.271788
2013-04-01    108.058798
2013-05-01    108.076974
2013-06-01    108.292294
2013-07-01    108.474990
2013-08-01    108.70

In [49]:
ax = cpi09.plot(title='CPI-2009')
#add_rec_bars(ax=ax)
sns.despine();

If you have trouble with the plotting, look into the `.fillna` and `.dropna` methods.
The next section explains what's going on.

If you finish this exercise early, check out the "Extra Exercises" section at the bottom of the notebook.

# Real GDP: Alignment

Let's say we want real GDP (adjusted for inflation).

\begin{equation}
    rGDP_t = \frac{GDP_t}{CPI_t}
\end{equation}

Problem: our CPI is monthly but GDP is quarterly. Also the two Series have different start and end points. If you didn't have automatic label alignment, you'd have to jump through hoops to select the correct subset of each series.


```python
# this is boolean indexing, we'll see more later
gdp / cpi09[(cpi.index.month % 3 == 1) & (cpi09.index.year <= 2014)]
```

... but that's unneccesary. The operations will automatically align for you.

In [50]:
rgdp = (gdp / cpi09)
rgdp

DATE
1947-01-01     24.283366
1947-02-01           NaN
1947-03-01           NaN
1947-04-01     24.021490
1947-05-01           NaN
1947-06-01           NaN
1947-07-01     24.139732
1947-08-01           NaN
1947-09-01           NaN
1947-10-01     24.378517
1947-11-01           NaN
1947-12-01           NaN
1948-01-01     24.120403
1948-02-01           NaN
1948-03-01           NaN
1948-04-01     24.582157
1948-05-01           NaN
1948-06-01           NaN
1948-07-01     24.578207
1948-08-01           NaN
1948-09-01           NaN
1948-10-01     24.775114
1948-11-01           NaN
1948-12-01           NaN
1949-01-01     24.611041
1949-02-01           NaN
1949-03-01           NaN
1949-04-01     24.371747
1949-05-01           NaN
1949-06-01           NaN
                 ...    
2012-12-01           NaN
2013-01-01    152.608507
2013-02-01           NaN
2013-03-01           NaN
2013-04-01    153.077771
2013-05-01           NaN
2013-06-01           NaN
2013-07-01    154.407021
2013-08-01          

The `NaN`s are missing value indicators. `NaN`s can crop up for many reasons, but in this case it's because the labels didn't overlap perfectly.

Binary operations (addition, multiplication, ...) consists of two steps.

1. align by label
2. operation

In [51]:
l, r = gdp.align(cpi09)
side_by_side(l.head(), r.head())


,GDP
DATE,
1947-01-01 00:00:00,243.1
1947-02-01 00:00:00,nan
1947-03-01 00:00:00,nan
1947-04-01 00:00:00,246.3
1947-05-01 00:00:00,nan
,CPIAUCSL
DATE,
1947-01-01 00:00:00,10.011
1947-02-01 00:00:00,10.0762


While binary operations *propogate* missing values, aggregations like `sum` or `mean`, will ignore them.
Pandas provides methods for detecting missing data `.isnull`, filling missing data `.fillna`,
or you can dropping it, which is what I'll do here.

In [55]:
rgdp.sort_values().isnull().count()

821

Pandas methods are non-mutating by default. This means that even though I called `.dropna()` above, `rgdp` still has missing values.

In [ ]:
rgdp

To capture the change, assign a name to the result. In this case, I just the same name `rgdp`.

In [ ]:
real_gdp = rgdp.dropna()
real_gdp.head()

In [ ]:
ax = gdp.pct_change().plot(
    figsize=(15, 5), label='GDP', legend=True, linewidth=2
)
real_gdp.pct_change().plot(
    ax=ax, label='real GDP', legend=True, linewidth=2
)
add_rec_bars(ax)
sns.despine()

# Recap

DataFrames allow you to rapidly explore data, iterating on ideas.

- `ufunc`s work as expected
- operations between pandas objects automatically align on labels

<div class="alert alert-info">
    <b>Extra Exercises</b>
</div>

Explore [FRED](https://fred.stlouisfed.org) for additional datasets.

See if you can produce an updated version of this graph from [*Calculated Risk*](Calculated Risk: BLS: Jobs Openings at 5.0 million in December, Up 28% Year-over-year).

In [ ]:
from IPython.display import Image
Image(url='http://1.bp.blogspot.com/-Q60opady2iQ/VNoeGepLMWI/AAAAAAAAiQc/PCA1LfFJFCg/s1600/JOLTSDec2014.PNG')

This shows the components of turnover in the labor market. People leave jobs for two reasons: 1. Quits (light blue) or 2. Layoffs, discharges, or other (red).

Companies post job openings (yellow) and fill some number of those (Blue).

The difference between the openings and hires represents the change in the stock of open positions, and measures the slackness of the labor market.

The difference between the Hires and the sum of Quites and Layoffs / Discharges represents the change in the employed labor force (though the unemployment rate is calculated differently).

To get you started, the FRED series codes are

- [JTSJOL](https://fred.stlouisfed.org/series/JTSJOL): Openings
- [JTSQUL](https://fred.stlouisfed.org/series/JTSQUL): Quits
- [JTSHIL](https://fred.stlouisfed.org/series/JTSHIL): Hires
- [JTSLDL](https://fred.stlouisfed.org/series/JTSLDL): Layoffs

Use the `pandas_datareader.data.DataReader` class, which accepts a list of symbols, and a `data_source` ('`fred'`).

As a hint, you can use pandas' `.plot.area()` method for the Quits and Layoffs series. That handles all the stacking for you.

In [ ]:
series = ['JTSJOL', 'JTSQUL', 'JTSHIL', 'JTSLDL']
names = ['openings', 'quits', 'hires', 'layoffs']
from pandas_datareader.data import DataReader

In [ ]:
res = DataReader(series, start='2000-01-01', data_source="fred")

In [ ]:
res = res.rename(columns=dict(zip(series, names)))

In [ ]:
ax = res[['quits', 'layoffs']].plot.area()
res[['hires', 'openings']].plot(ax=ax)

In [ ]:
k